In [5]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [7]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [8]:
data = pd.read_csv('2003_01_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t),Rio De Janeiro - Consumo de Cimento (t)
0,2003,0.773489,3.641734,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.356512,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,2377.549731,3021.269
1,2004,0.774193,3.879151,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.282350,3.631565e+08,1.820415e+07,22.209810,3.487366e+08,2400.424606,3092.530
2,2005,0.774914,3.969821,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.259763,3.674546e+08,1.823716e+07,22.259791,3.497876e+08,2511.833086,3324.329
3,2006,0.775653,3.958649,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.165392,3.711498e+08,1.831286e+07,22.286909,3.503984e+08,2676.431443,3665.682
4,2007,0.776412,3.899249,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.139058,3.742425e+08,1.841057e+07,22.295610,3.506279e+08,2858.554649,4107.097
5,2008,0.777194,3.862900,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.106163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,3010.012911,4300.676
6,2009,0.778002,3.934097,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.134661,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,3108.577595,3966.930
7,2010,0.778838,4.053038,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.098147,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,3181.826392,4199.911
8,2011,0.779704,4.190909,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.084479,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,3209.638424,4390.254
9,2012,0.780264,4.443161,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.891336,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,3199.169780,4733.632


In [9]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,0.773489,3.641734,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.356512,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,2377.549731
1,0.774193,3.879151,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.282350,3.631565e+08,1.820415e+07,22.209810,3.487366e+08,2400.424606
2,0.774914,3.969821,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.259763,3.674546e+08,1.823716e+07,22.259791,3.497876e+08,2511.833086
3,0.775653,3.958649,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.165392,3.711498e+08,1.831286e+07,22.286909,3.503984e+08,2676.431443
4,0.776412,3.899249,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.139058,3.742425e+08,1.841057e+07,22.295610,3.506279e+08,2858.554649
5,0.777194,3.862900,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,8.106163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,3010.012911
6,0.778002,3.934097,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.134661,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,3108.577595
7,0.778838,4.053038,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,8.098147,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,3181.826392
8,0.779704,4.190909,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,8.084479,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,3209.638424
9,0.780264,4.443161,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,9.891336,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,3199.169780


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.530
1     3324.329
2     3665.682
3     4107.097
4     4300.676
5     3966.930
6     4199.911
7     4390.254
8     4733.632
9     4864.477
10    4454.468
11    3639.072
12    2811.648
13    2284.918
14    2391.212
15    2715.843
16    2935.607
17    2829.682
18    2682.995
19         NaN
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,-1.659894,-1.327573,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.800768,-1.715997,0.180107,0.159367,-1.316734,-1.124902
1,-1.463956,-0.955104,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.837605,-1.458046,0.330237,0.527554,-0.397385,-1.054751
2,-1.263651,-0.812857,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.848825,-1.227711,0.402247,0.806003,0.268145,-0.713089
3,-1.058300,-0.830384,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.895700,-1.029691,0.567418,0.957081,0.654873,-0.208309
4,-0.847225,-0.923574,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.908780,-0.863954,0.780586,1.005557,0.800196,0.350216
5,-0.629747,-0.980599,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.925120,-0.721075,0.948711,1.057989,0.886709,0.814700
6,-0.405189,-0.868902,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.910964,-0.640511,1.130083,0.891357,0.531630,1.116973
7,-0.172871,-0.682303,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.929101,-0.188552,1.138875,0.953673,1.175773,1.341608
8,0.067883,-0.466005,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.935890,0.210381,0.953179,0.846349,1.469509,1.426900
9,0.223642,-0.070264,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-0.038401,0.531697,0.727660,0.496052,1.282377,1.394795


In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     3092.530
1     3324.329
2     3665.682
3     4107.097
4     4300.676
5     3966.930
6     4199.911
7     4390.254
8     4733.632
9     4864.477
10    4454.468
11    3639.072
12    2811.648
13    2284.918
14    2391.212
15    2715.843
16    2935.607
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [13]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
17,-0.064841,1.959291,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.166651,0.74046,-1.172346,-1.219689,-1.234773,-0.016416


In [14]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2829.682
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [15]:
train_input.iloc[-(len(train_input)//5):]

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
14,1.333259,1.260323,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.370913,1.150765,-1.658434,-1.458301,-1.186853,-1.203759
15,1.162836,1.294615,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.295665,1.170237,-1.626613,-1.265974,-0.810360,-0.575010
16,0.623127,1.547309,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.279383,1.028274,-1.514053,-1.285519,-1.005031,-0.242451


In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2543825806, 4066804048, 3804102612, 1952853287, 2193701979, 2220607956, 1137785576, 4248362760, 2852474145, 1943390385, 3285191425, 3923042360, 634428376, 684790123, 2685482397, 3062326778, 1850917052, 4128916168, 235563120, 4002006574, 824090153, 4281224049, 2032260940, 1998364714, 539414662, 1418753346, 1967147923, 280911135, 1029603066, 3464343791, 2382202511, 1616474762, 2198167996, 1207403775, 3443347465, 3424681895, 1560625821, 428592134, 701359347, 3951272911, 4235816649, 1602570454, 3440077916, 1497758189, 2095463831, 1051716363, 1543863842, 1243621389, 1970327783, 2487046707, 3507827371, 3809075055, 2268925209, 3956217946, 3070942434, 2015680981, 2679904498, 3386857680, 3500204854, 4264064722, 164347394, 874813096, 1179009697, 3287798587, 2701918525, 1711590338, 3003784734, 923427414, 2934290757, 970929151, 658001985, 659317811, 1708499738, 3370389065, 1200526252, 517404059, 1725462387, 1213519566, 3444238127, 1593338396, 96427139, 1781254536, 1333249735, 1987570828, 23209622

val_loss: 34954.515625


Step: 88 ___________________________________________
val_loss: 47663.609375


Step: 89 ___________________________________________
val_loss: 22951.177734375


Step: 90 ___________________________________________
val_loss: 50009.859375


Step: 91 ___________________________________________
val_loss: 28269.078125


Step: 92 ___________________________________________
val_loss: 19321.466796875


Step: 93 ___________________________________________
val_loss: 90825.4140625


Step: 94 ___________________________________________
val_loss: 2648.40673828125
winner_seed: 1827875205


Step: 95 ___________________________________________
val_loss: 49422.63671875


Step: 96 ___________________________________________
val_loss: 27661.068359375


Step: 97 ___________________________________________
val_loss: 50128.14453125


Step: 98 ___________________________________________
val_loss: 50005.671875


Step: 99 ___________________________________________
val_loss: 22927.3125



In [21]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 447ms/step - loss: 13886647.0000 - val_loss: 6981772.5000
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 13673311.0000 - val_loss: 6599098.5000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 14619851.0000 - val_loss: 15457308.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 10625706.0000 - val_loss: 21747770.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 11427546.0000 - val_loss: 17745366.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 10276655.0000 - val_loss: 12543812.0000
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 10272854.0000 - val_loss: 9819348.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 8537345.0000 - val_loss: 9023601.0000
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 9552576.0000 - val

1/1 [==============================] - 0s 13ms/step - loss: 3494910.7500 - val_loss: 2313284.2500
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 3220594.5000 - val_loss: 2988389.0000
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 3798959.2500 - val_loss: 899273.8125
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 3339538.0000 - val_loss: 3132569.2500
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 2614444.0000 - val_loss: 2682214.7500
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 2929514.7500 - val_loss: 2384938.2500
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 2767071.0000 - val_loss: 2561671.7500
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 3082485.7500 - val_loss: 1896863.3750
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 2881529.0000 - val_loss: 2715083.7500

1/1 [==============================] - 0s 13ms/step - loss: 815237.5000 - val_loss: 447347.8438
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 665961.8750 - val_loss: 392670.4062
Epoch 147/10000
1/1 [==============================] - 0s 14ms/step - loss: 550807.1875 - val_loss: 455899.2500
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 847921.6875 - val_loss: 535119.1875
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 697099.3125 - val_loss: 536903.1875
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 705696.0000 - val_loss: 462727.0938
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 976744.6875 - val_loss: 238257.9531
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 652251.5000 - val_loss: 303295.6562
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 758787.5000 - val_loss: 226426.8750
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 524728.8125 - val_loss: 83189.6719
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 421013.9375 - val_loss: 88484.9766
Epoch 220/10000
1/1 [==============================] - 0s 13ms/step - loss: 285578.5938 - val_loss: 100724.9453
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 657690.1250 - val_loss: 138370.2812
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 339333.1250 - val_loss: 85124.2109
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 352584.2500 - val_loss: 62419.4180
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 367815.9688 - val_loss: 70973.6250
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 393837.0312 - val_loss: 55455.1914
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 494739.4688 - val_loss: 52000.9492
Epoch 227/10000

Epoch 292/10000
1/1 [==============================] - 0s 13ms/step - loss: 173482.1719 - val_loss: 50748.2773
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 392998.4375 - val_loss: 54382.0664
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 211490.4375 - val_loss: 53690.7383
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 225699.7969 - val_loss: 61534.9688
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 400188.4375 - val_loss: 399959.5000
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 596105.0625 - val_loss: 58081.8555
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 232481.7812 - val_loss: 60547.9531
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 333810.6250 - val_loss: 83976.7031
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 226728.6250 - val_loss: 52444.8633


...

Epoch 1027/10000
1/1 [==============================] - 0s 14ms/step - loss: 128306.3359 - val_loss: 80410.3516
Epoch 1028/10000
1/1 [==============================] - 0s 14ms/step - loss: 86126.8281 - val_loss: 236445.6406
Epoch 1029/10000
1/1 [==============================] - 0s 13ms/step - loss: 90760.2891 - val_loss: 111243.5078
Epoch 1030/10000
1/1 [==============================] - 0s 13ms/step - loss: 101259.0938 - val_loss: 61941.9414
Epoch 1031/10000
1/1 [==============================] - 0s 14ms/step - loss: 124824.9531 - val_loss: 102055.7109
Epoch 1032/10000
1/1 [==============================] - 0s 14ms/step - loss: 50461.8047 - val_loss: 125877.7734
Epoch 1033/10000
1/1 [==============================] - 0s 13ms/step - loss: 93945.0078 - val_loss: 61446.6445
Epoch 1034/10000
1/1 [==============================] - 0s 14ms/step - loss: 90345.0625 - val_loss: 82121.9531
Epoch 1035/10000
1/1 [==============================] - 0s 13ms/step - loss: 107553.8828 - val_loss: 9481

1/1 [==============================] - 0s 13ms/step - loss: 80801.4844 - val_loss: 13184.2812
Epoch 1101/10000
1/1 [==============================] - 0s 13ms/step - loss: 71344.2344 - val_loss: 19025.0918
Epoch 1102/10000
1/1 [==============================] - 0s 13ms/step - loss: 33386.6367 - val_loss: 16883.1621
Epoch 1103/10000
1/1 [==============================] - 0s 13ms/step - loss: 61002.7188 - val_loss: 12479.0674
Epoch 1104/10000
1/1 [==============================] - 0s 14ms/step - loss: 121319.0078 - val_loss: 23226.7168
Epoch 1105/10000
1/1 [==============================] - 0s 13ms/step - loss: 75409.4297 - val_loss: 11895.7080
Epoch 1106/10000
1/1 [==============================] - 0s 13ms/step - loss: 101562.6641 - val_loss: 18893.2051
Epoch 1107/10000
1/1 [==============================] - 0s 13ms/step - loss: 76045.5547 - val_loss: 14572.2354
Epoch 1108/10000
1/1 [==============================] - 0s 13ms/step - loss: 72391.1875 - val_loss: 13973.8984
Epoch 1109/10000

Epoch 1174/10000
1/1 [==============================] - 0s 14ms/step - loss: 51952.2227 - val_loss: 64310.5781
Epoch 1175/10000
1/1 [==============================] - 0s 14ms/step - loss: 75965.0000 - val_loss: 69963.6875
Epoch 1176/10000
1/1 [==============================] - 0s 13ms/step - loss: 110215.3984 - val_loss: 65317.7617
Epoch 1177/10000
1/1 [==============================] - 0s 13ms/step - loss: 53438.0078 - val_loss: 27395.3926
Epoch 1178/10000
1/1 [==============================] - 0s 13ms/step - loss: 117467.9062 - val_loss: 12814.9385
Epoch 1179/10000
1/1 [==============================] - 0s 13ms/step - loss: 61819.6992 - val_loss: 57292.6914
Epoch 1180/10000
1/1 [==============================] - 0s 13ms/step - loss: 105307.8203 - val_loss: 52986.5039
Epoch 1181/10000
1/1 [==============================] - 0s 13ms/step - loss: 75212.2266 - val_loss: 26844.5469
Epoch 1182/10000
1/1 [==============================] - 0s 13ms/step - loss: 73777.5625 - val_loss: 49187.113

1/1 [==============================] - 0s 13ms/step - loss: 129242.0312 - val_loss: 24878.4746
Epoch 1248/10000
1/1 [==============================] - 0s 13ms/step - loss: 117593.2969 - val_loss: 14537.4404
Epoch 1249/10000
1/1 [==============================] - 0s 14ms/step - loss: 102198.8672 - val_loss: 32451.9199
Epoch 1250/10000
1/1 [==============================] - 0s 13ms/step - loss: 99812.5859 - val_loss: 14620.3994
Epoch 1251/10000
1/1 [==============================] - 0s 13ms/step - loss: 124601.6875 - val_loss: 20481.4453
Epoch 1252/10000
1/1 [==============================] - 0s 13ms/step - loss: 56860.1523 - val_loss: 31970.9844
Epoch 1253/10000
1/1 [==============================] - 0s 13ms/step - loss: 109503.6016 - val_loss: 32115.2441
Epoch 1254/10000
1/1 [==============================] - 0s 13ms/step - loss: 94096.4844 - val_loss: 32773.6562
Epoch 1255/10000
1/1 [==============================] - 0s 14ms/step - loss: 139931.6562 - val_loss: 37309.0469
Epoch 1256/1

1/1 [==============================] - 0s 13ms/step - loss: 81551.1484 - val_loss: 60360.1562
Epoch 1321/10000
1/1 [==============================] - 0s 13ms/step - loss: 101285.6250 - val_loss: 66644.2969
Epoch 1322/10000
1/1 [==============================] - 0s 14ms/step - loss: 86506.0781 - val_loss: 68377.8984
Epoch 1323/10000
1/1 [==============================] - 0s 14ms/step - loss: 143804.3594 - val_loss: 93341.4141
Epoch 1324/10000
1/1 [==============================] - 0s 13ms/step - loss: 66280.0078 - val_loss: 120859.6562
Epoch 1325/10000
1/1 [==============================] - 0s 13ms/step - loss: 83592.7109 - val_loss: 84044.3906
Epoch 1326/10000
1/1 [==============================] - 0s 13ms/step - loss: 91936.0625 - val_loss: 91416.9297
Epoch 1327/10000
1/1 [==============================] - 0s 13ms/step - loss: 104008.8047 - val_loss: 114404.5391
Epoch 1328/10000
1/1 [==============================] - 0s 13ms/step - loss: 112282.1719 - val_loss: 86094.6250
Epoch 1329/1

In [22]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[2829.682](test_target) - [[2770.5408]](prediction) = 59.14122851562479


In [23]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [24]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,-1.485690,-2.292367,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.848413,-1.616056,-1.458906,-2.229486,-2.156922,-1.297696
1,-1.082662,-0.183669,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.011304,-1.049130,-1.024912,-0.937706,-0.949370,-1.221210
2,-0.670651,0.621648,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.756350,-0.542899,-0.816746,0.039227,-0.075205,-0.848693
3,-0.248261,0.522424,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.308873,-0.107689,-0.339276,0.569279,0.432757,-0.298325
4,0.185903,-0.005163,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.606118,0.256567,0.276944,0.739358,0.623637,0.310641
5,0.633236,-0.328003,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.977421,0.570587,0.762956,0.923316,0.737271,0.817073
6,1.095133,0.304359,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.655750,0.747652,1.287262,0.338688,0.270880,1.146644
7,1.572990,1.360772,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.067906,1.740969,1.312678,0.557324,1.116953,1.391566


train_target:


,Rio De Janeiro - Consumo de Cimento (t)
0,3092.530
1,3324.329
2,3665.682
3,4107.097
4,4300.676
5,3966.930
6,4199.911
7,4390.254


test_input:


...

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
17,-0.064841,1.959291,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.166651,0.74046,-1.172346,-1.219689,-1.234773,-0.016416


test_target:


,Rio De Janeiro - Consumo de Cimento (t)
17,2829.682


1/1 [==============================] - 0s 34ms/step
Error: 59.14122851562479




[4363.5478515625,
 4415.87548828125,
 4580.23681640625,
 4053.3564453125,
 3864.84130859375,
 3326.161865234375,
 2867.95458984375,
 3021.00439453125,
 2787.881591796875,
 2770.540771484375]

In [25]:
display(targets)
display(predictions)

[4733.6320000000005,
 4864.477,
 4454.468,
 3639.072,
 2811.648,
 2284.918,
 2391.212,
 2715.843,
 2935.607,
 2829.682]

[4363.5478515625,
 4415.87548828125,
 4580.23681640625,
 4053.3564453125,
 3864.84130859375,
 3326.161865234375,
 2867.95458984375,
 3021.00439453125,
 2787.881591796875,
 2770.540771484375]

In [26]:
mae = mean_absolute_error(predictions, targets)
mae

444.19467167968753

In [27]:
porcentage = mae/np.mean(targets)
porcentage

0.13196295155992135